In [ ]:
from kafka import KafkaConsumer, KafkaProducer
import json 
from jsonschema import validate, ValidationError

with open('sensor_schema.json') as f:
    schema = json.load(f)

consumer = KafkaConsumer(
    'raw_iot',
    bootstrap_servers = 'kafka_streaming_lab:9092',
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id = 'validator',
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)

producer = KafkaProducer(
    bootstrap_servers = 'kafka_streaming_lab:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

print("Validator started...")


for msg in consumer:
    data = msg.value
    try:
        validate(instance = data, schema = schema)
        producer.send('iot_data', value=data)
        print(f"Valid message: {data}")
    except ValidationError as e:
        data['error_message'] = e.message
        print(f"Invalid message: {data}")
        producer.send('error_log', value=data)



Validator started...
Invalid message: {'device_id': 'sensor-0', 'temperature': -9.86, 'humidity': 116.28, 'event_timestamp': 1747327889.3125596, 'error_message': '-9.86 is less than the minimum of 0'}
Invalid message: {'device_id': 'sensor-1', 'temperature': 59.65, 'humidity': 40.09, 'event_timestamp': 1747327889.8158782, 'error_message': '59.65 is greater than the maximum of 50'}
Invalid message: {'device_id': 'sensor-2', 'temperature': 42.22, 'humidity': 109.49, 'event_timestamp': 1747327890.3174708, 'error_message': '109.49 is greater than the maximum of 100'}
Invalid message: {'device_id': 'sensor-0', 'temperature': 25.29, 'humidity': 119.03, 'event_timestamp': 1747327890.8186505, 'error_message': '119.03 is greater than the maximum of 100'}
Valid message: {'device_id': 'sensor-1', 'temperature': 45.8, 'humidity': 66.9, 'event_timestamp': 1747327891.320236}
Invalid message: {'device_id': 'sensor-2', 'temperature': 52.02, 'humidity': 99.2, 'event_timestamp': 1747327891.822199, 'erro